# The Battle of the Neighborhoods


## 1. Description of the Problem and Discussion of the Background 

### Selecting the best neighborhood to relocate.

Currently, I am planning to relocate to Mandaluyong City, Philippines. Also, I have my preferences of in terms of the place and its venues. Hence, clustering the barangays in Mandaluyong City will help me pick the best clusters and choose among the barangays on where to relocate.

## 2. Initial Data Preparation

1. Get The names of barangays in Mandaluyong City from Wikipedia 
2. Processing the information From Wikipedia To make necessary lists
3. Get the coordinates of the barangays
4. Check and compare with Google Search and refine if necessary

## 3. Exploratory Data Analysis

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import csv

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

### Get The Names of Wards, Major Districts and Population from Wikipedia

In [29]:
# Get data from the given link
url = 'https://en.wikipedia.org/wiki/Mandaluyong'
wiki = requests.get(url).text

# Use beautiful soup to parse the HTML/XML codes
soup = BeautifulSoup(wiki,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Mandaluyong - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"479942a9-580c-4168-97d0-e339d3a953ef","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Mandaluyong","wgTitle":"Mandaluyong","wgCurRevisionId":964223428,"wgRevisionId":964223428,"wgArticleId":359686,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Use dmy dates from July 2019","Articles with short description","Pages using infobox settlement with possible motto list",

In [30]:
table = soup.find('table',{'class':'sortable wikitable'})
table

<table class="sortable wikitable">
<tbody><tr>
<th>District
</th>
<th>Barangay
</th>
<th>Barangay<br/>Chairman
</th>
<th>Land Area<br/>(<a href="/wiki/Hectare" title="Hectare">has</a>.)<sup class="reference" id="cite_ref-14"><a href="#cite_note-14">[14]</a></sup>
</th>
<th>Population<br/>(2007)<sup class="reference" id="cite_ref-census07_15-0"><a href="#cite_note-census07-15">[15]</a></sup>
</th>
<th>Population<br/>(2010)<sup class="reference" id="cite_ref-2010_census_16-0"><a href="#cite_note-2010_census-16">[16]</a></sup>
</th>
<th>Population<br/>(2015)<sup class="reference" id="cite_ref-17"><a href="#cite_note-17">[17]</a></sup>
</th></tr>
<tr>
<td>1
</td>
<td><a class="new" href="/w/index.php?title=Addition_Hills,_Mandaluyong&amp;action=edit&amp;redlink=1" title="Addition Hills, Mandaluyong (page does not exist)">Addition Hills</a>
</td>
<td>Carlito Cernal
</td>
<td>162.00
</td>
<td>81,221
</td>
<td>86,731
</td>
<td>99,058
</td></tr>
<tr>
<td>1
</td>
<td><a class="new" href="/w/ind

In [33]:
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['District', 'Barangay', 'Barangay_Chairman', 'Land_Area', 'Population1', 'Population2', 'Population3'])
df = df[~df['District'].isnull()] #to filter

df.head(11)

,District,Barangay,Barangay_Chairman,Land_Area,Population1,Population2,Population3
1,1,Addition Hills,Carlito Cernal,162.00,"81,221","86,731","99,058"
2,1,Bagong Silang,Kristofer Dominguez,14.26,"3,747","4,652","5,572"
3,2,Barangka Drive,Darwin Fernandez,24.54,"12,134","12,227","13,310"
4,2,Barangka Ibaba,Edwin Santa Maria,16.92,"9,372","9,241","9,540"
5,2,Barangka Ilaya,Joselito Pangilinan,47.45,"4,185","5,049","17,896"
6,2,Barangka Itaas,Dannie Ocampo,17.21,"11,212","11,061","11,252"
7,2,Buayang Bato,Reynaldo Nobela,7.26,999,"1,340","1,782"
8,1,Burol,"Ernesto Santos, Jr.",2.78,"2,322","2,606","2,740"
9,1,Daang Bakal,Richard Bassig,17.34,"2,980","3,931","3,660"
10,1,Hagdan Bato Itaas,Edmon Espiritu,18.36,"9,431","10,102","10,314"


In [34]:
df_new = df.drop(['District', 'Barangay_Chairman', 'Land_Area', 'Population1', 'Population2', 'Population3'], axis=1)
df_new

,Barangay
1,Addition Hills
2,Bagong Silang
3,Barangka Drive
4,Barangka Ibaba
5,Barangka Ilaya
6,Barangka Itaas
7,Buayang Bato
8,Burol
9,Daang Bakal
10,Hagdan Bato Itaas


In [51]:
df_new_city = df_new['Barangay'] + ", Mandaluyong"

In [52]:
df_fin = pd.DataFrame(df_new_city)
df_fin

,Barangay
1,"Addition Hills, Mandaluyong"
2,"Bagong Silang, Mandaluyong"
3,"Barangka Drive, Mandaluyong"
4,"Barangka Ibaba, Mandaluyong"
5,"Barangka Ilaya, Mandaluyong"
6,"Barangka Itaas, Mandaluyong"
7,"Buayang Bato, Mandaluyong"
8,"Burol, Mandaluyong"
9,"Daang Bakal, Mandaluyong"
10,"Hagdan Bato Itaas, Mandaluyong"


In [54]:
geolocator = Nominatim(user_agent="Manda_explorer")
df_new['Barangay_Coor']= df_new['Barangay'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_new[['Latitude', 'Longitude']] = df_new['Barangay_Coor'].apply(pd.Series)

In [50]:
df_new

,Barangay,Barangay_Coor,Latitude,Longitude
1,Addition Hills,"(14.5942519, 121.0407231)",14.594252,121.040723
2,Bagong Silang,"(7.731389, 122.900556)",7.731389,122.900556
3,Barangka Drive,"(14.5717991, 121.0375077)",14.571799,121.037508
4,Barangka Ibaba,"(14.5703077, 121.0390332)",14.570308,121.039033
5,Barangka Ilaya,"(14.57253465, 121.04915867558185)",14.572535,121.049159
6,Barangka Itaas,"(14.5709691, 121.0419178)",14.570969,121.041918
7,Buayang Bato,"(14.5727521, 121.0534114)",14.572752,121.053411
8,Burol,"(15.5515549, 120.992231)",15.551555,120.992231
9,Daang Bakal,"(14.5935435, 121.02806473670462)",14.593543,121.028065
10,Hagdan Bato Itaas,"(14.588103, 121.03264105938521)",14.588103,121.032641


In [64]:
df_drop = df_new.drop(df_new[(df_new.Latitude < 14) | (df_new.Latitude > 14.90)].index)
df_drop.reset_index(inplace=True)
df_drop.drop(columns=['index'], axis=1, inplace=True)
df_drop

,Barangay,Barangay_Coor,Latitude,Longitude
0,Addition Hills,"(14.5942519, 121.0407231)",14.594252,121.040723
1,Barangka Drive,"(14.5717991, 121.0375077)",14.571799,121.037508
2,Barangka Ibaba,"(14.5703077, 121.0390332)",14.570308,121.039033
3,Barangka Ilaya,"(14.57253465, 121.04915867558185)",14.572535,121.049159
4,Barangka Itaas,"(14.5709691, 121.0419178)",14.570969,121.041918
5,Buayang Bato,"(14.5727521, 121.0534114)",14.572752,121.053411
6,Daang Bakal,"(14.5935435, 121.02806473670462)",14.593543,121.028065
7,Hagdan Bato Itaas,"(14.588103, 121.03264105938521)",14.588103,121.032641
8,Hagdan Bato Libis,"(14.58856675, 121.03296661147283)",14.588567,121.032967
9,Harapin Ang Bukas,"(14.59089995, 121.02805568457521)",14.590900,121.028056


### Get Mandaluyong City's coordinates

In [65]:
address = 'Mandaluyong City'

geolocator = Nominatim(user_agent="Manda_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Mandaluyong City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Mandaluyong City are 14.5777807, 121.0332295.


### Plot the barangays in a map

In [72]:
import folium
# # create map of the barangays in Mandaluyong City using latitude and longitude values
Manda_brgys= folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(df_drop['Latitude'], df_drop['Longitude'], 
                           df_drop['Barangay']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='magenta',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Manda_brgys)  
Manda_brgys

### Explore the nearby venues of each barangay

In [77]:
CLIENT_ID = 'STQ3OQSRT4NLZIA5GTAPLBKTZJGVVOVTIVG05NPRVHWY2AFR' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'LH1GKDNLU4QDKBI0RDYE1PYQBNLRRLGQDMI4I1XP2OEDZYHN' #'your-client-secret' # your Foursquare Secret
VERSION = '20190325' # Foursquare API version
LIMIT = 100
print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
My CLIENT_ID: STQ3OQSRT4NLZIA5GTAPLBKTZJGVVOVTIVG05NPRVHWY2AFR
My CLIENT_SECRET:LH1GKDNLU4QDKBI0RDYE1PYQBNLRRLGQDMI4I1XP2OEDZYHN


In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
Manda_Venues = getNearbyVenues(names=df_drop['Barangay'],
                                   latitudes=df_drop['Latitude'],
                                   longitudes=df_drop['Longitude']
                                  )

Addition Hills
Barangka Drive
Barangka Ibaba
Barangka Ilaya
Barangka Itaas
Buayang Bato
Daang Bakal
Hagdan Bato Itaas
Hagdan Bato Libis
Harapin Ang Bukas
Highway Hills
Mabini–J.Rizal
Mauway
Namayan
New Zañiga
Old Zañiga
Wack-Wack Greenhills


In [83]:
print ("Shape of the Venues Dataframe: ", Manda_Venues.shape)
print(Manda_Venues.head(3))
print(Manda_Venues.tail(3))

Shape of the Venues Dataframe:  (330, 7)
    Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0  Addition Hills               14.594252               121.040723   
1  Addition Hills               14.594252               121.040723   
2  Addition Hills               14.594252               121.040723   

                      Venue  Venue Latitude  Venue Longitude  \
0       Francesco's Kitchen       14.592989       121.041251   
1      Tomo Japanese Dining       14.592348       121.038498   
2  Mala Tang Hotpot Kitchen       14.596975       121.039142   

        Venue Category  
0   Italian Restaurant  
1  Japanese Restaurant  
2   Chinese Restaurant  
            Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
327  Wack-Wack Greenhills                14.59242               121.053835   
328  Wack-Wack Greenhills                14.59242               121.053835   
329  Wack-Wack Greenhills                14.59242               121.053835   

        

In [84]:
Manda_Venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 7 columns):
Neighbourhood              330 non-null object
Neighbourhood Latitude     330 non-null float64
Neighbourhood Longitude    330 non-null float64
Venue                      330 non-null object
Venue Latitude             330 non-null float64
Venue Longitude            330 non-null float64
Venue Category             330 non-null object
dtypes: float64(4), object(3)
memory usage: 18.1+ KB


In [88]:
Manda_Venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Addition Hills,29,29,29,29,29,29
Barangka Drive,16,16,16,16,16,16
Barangka Ibaba,6,6,6,6,6,6
Barangka Ilaya,23,23,23,23,23,23
Barangka Itaas,13,13,13,13,13,13
Buayang Bato,28,28,28,28,28,28
Daang Bakal,28,28,28,28,28,28
Hagdan Bato Itaas,17,17,17,17,17,17
Hagdan Bato Libis,18,18,18,18,18,18


In [89]:
print('There are {} uniques categories.'.format(len(Manda_Venues['Venue Category'].unique())))

There are 88 uniques categories.


In [90]:
# one hot encoding
Manda_onehot = pd.get_dummies(Manda_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Manda_onehot['Neighbourhood'] = Manda_Venues['Neighbourhood'] 

# move neighborhood column to the first column
Manda_fixed_columns = [Manda_onehot.columns[-1]] + list(Manda_onehot.columns[:-1])
Manda_onehot = Manda_onehot[Manda_fixed_columns]

Manda_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Climbing Gym,Clothing Store,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Drugstore,Fast Food Restaurant,Filipino Restaurant,Food Court,Food Truck,Gastropub,Greek Restaurant,Grocery Store,Gym,Health & Beauty Service,Hobby Shop,Hotel,Ice Cream Shop,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Laundromat,Market,Massage Studio,Mediterranean Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Outdoor Event Space,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Snack Place,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Taco Place,Tailor Shop,Tea Room,Tennis Court,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [91]:
Manda_grouped = Manda_onehot.groupby('Neighbourhood').mean().reset_index()
Manda_grouped.head(20)

,Neighbourhood,American Restaurant,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Climbing Gym,Clothing Store,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Drugstore,Fast Food Restaurant,Filipino Restaurant,Food Court,Food Truck,Gastropub,Greek Restaurant,Grocery Store,Gym,Health & Beauty Service,Hobby Shop,Hotel,Ice Cream Shop,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Laundromat,Market,Massage Studio,Mediterranean Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Outdoor Event Space,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Snack Place,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Taco Place,Tailor Shop,Tea Room,Tennis Court,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Addition Hills,0.034483,0.000000,0.0,0.034483,0.068966,0.103448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.034483,0.103448,0.000000,0.000000,0.068966,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.034483,0.103448,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.0,0.00,0.034483,0.034483,0.034483
1,Barangka Drive,0.000000,0.000000,0.0,0.125000,0.000000,0.062500,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.0,0.00,0.000000,0.000000,0.000000
2,Barangka Ibaba,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000
3,Barangka Ilaya,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.043478,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.0,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000

In [92]:
num_top_venues = 10

for hood in Manda_grouped['Neighbourhood']:
    print("----"+hood+"----")
    Manda_temp = Manda_grouped[Manda_grouped['Neighbourhood'] == hood].T.reset_index()
    Manda_temp.columns = ['venue','freq']
    Manda_temp = Manda_temp.iloc[1:]
    Manda_temp['freq'] = Manda_temp['freq'].astype(float)
    Manda_temp = Manda_temp.round({'freq': 2})
    print(Manda_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Addition Hills----
                           venue  freq
0             Chinese Restaurant  0.10
1                            Bar  0.10
2            Japanese Restaurant  0.10
3                         Bakery  0.07
4                Bubble Tea Shop  0.07
5              Convenience Store  0.07
6                    Coffee Shop  0.07
7            American Restaurant  0.03
8          Vietnamese Restaurant  0.03
9  Vegetarian / Vegan Restaurant  0.03


----Barangka Drive----
                  venue  freq
0          Burger Joint  0.12
1             BBQ Joint  0.12
2                   Spa  0.12
3           Tailor Shop  0.06
4     Convenience Store  0.06
5  Fast Food Restaurant  0.06
6   Filipino Restaurant  0.06
7            Food Truck  0.06
8    Salon / Barbershop  0.06
9      Basketball Court  0.06


----Barangka Ibaba----
                  venue  freq
0                 Hotel  0.17
1                   Bar  0.17
2      Basketball Court  0.17
3            Taco Place  0.17
4  Fast Food Resta

In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Manda_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

Manda_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [95]:
Manda_neighbourhoods_venues_sorted['Neighbourhood'] = Manda_grouped['Neighbourhood']

for ind in np.arange(Manda_grouped.shape[0]):
    Manda_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manda_grouped.iloc[ind, :], num_top_venues)

Manda_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,Bar,Chinese Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Coffee Shop,Convenience Store,Yoga Studio,Movie Theater,BBQ Joint
1,Barangka Drive,Spa,BBQ Joint,Burger Joint,Filipino Restaurant,Massage Studio,Salon / Barbershop,Laundromat,Fast Food Restaurant,Convenience Store,Bar
2,Barangka Ibaba,Hotel,Bar,Taco Place,Basketball Court,Food Truck,Fast Food Restaurant,Donut Shop,Creperie,Dance Studio,Deli / Bodega
3,Barangka Ilaya,Coffee Shop,Filipino Restaurant,Sandwich Place,Hotel,Multiplex,Dance Studio,Massage Studio,Chinese Restaurant,Café,Market
4,Barangka Itaas,Fast Food Restaurant,Tea Room,Filipino Restaurant,Bar,Diner,Burger Joint,Bookstore,Bed & Breakfast,Convenience Store,Tailor Shop
5,Buayang Bato,Coffee Shop,Sandwich Place,Filipino Restaurant,Japanese Restaurant,Yoga Studio,Bakery,Café,Chinese Restaurant,Dance Studio,Dessert Shop
6,Daang Bakal,Fast Food Restaurant,Convenience Store,Filipino Restaurant,Chinese Restaurant,Salon / Barbershop,Snack Place,Pharmacy,Bakery,Pizza Place,Supermarket
7,Hagdan Bato Itaas,Convenience Store,Fast Food Restaurant,Food Court,Coffee Shop,Diner,Shopping Mall,Burger Joint,Spa,Korean Restaurant,Café
8,Hagdan Bato Libis,Fast Food Restaurant,Convenience Store,Korean Restaurant,Café,Diner,Shopping Mall,Burger Joint,Coffee Shop,Spa,Breakfast Spot
9,Harapin Ang Bukas,Fast Food Restaurant,Chinese Restaurant,Café,Filipino Restaurant,Supermarket,Snack Place,Salon / Barbershop,Convenience Store,Donut Shop,Bookstore


In [96]:
# set number of clusters
kclusters = 3

Manda_grouped_clustering = Manda_grouped.drop('Neighbourhood', 1)

# run k-means clustering
Manda_kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(Manda_grouped_clustering)

# check cluster labels generated for each row in the dataframe
Manda_kmeans.labels_[0:5] 

array([0, 0, 0, 0, 0], dtype=int32)

Rename column name of df_drop and drop Barangay_Coor

In [101]:
df_drop.rename(columns={"Barangay": "Neighbourhood"}, inplace=True)
df_drop.drop(['Barangay_Coor'], axis=1, inplace=True)
df_drop

,Neighbourhood,Latitude,Longitude
0,Addition Hills,14.594252,121.040723
1,Barangka Drive,14.571799,121.037508
2,Barangka Ibaba,14.570308,121.039033
3,Barangka Ilaya,14.572535,121.049159
4,Barangka Itaas,14.570969,121.041918
5,Buayang Bato,14.572752,121.053411
6,Daang Bakal,14.593543,121.028065
7,Hagdan Bato Itaas,14.588103,121.032641
8,Hagdan Bato Libis,14.588567,121.032967
9,Harapin Ang Bukas,14.590900,121.028056


In [102]:
# add clustering labels
Manda_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', Manda_kmeans.labels_)

Manda_merged = df_drop

# merge york_neighbourhoods_venues_sorted with df5 to add latitude/longitude for each neighborhood
Manda_merged = Manda_merged.join(Manda_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Manda_merged

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,14.594252,121.040723,0,Bar,Chinese Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Coffee Shop,Convenience Store,Yoga Studio,Movie Theater,BBQ Joint
1,Barangka Drive,14.571799,121.037508,0,Spa,BBQ Joint,Burger Joint,Filipino Restaurant,Massage Studio,Salon / Barbershop,Laundromat,Fast Food Restaurant,Convenience Store,Bar
2,Barangka Ibaba,14.570308,121.039033,0,Hotel,Bar,Taco Place,Basketball Court,Food Truck,Fast Food Restaurant,Donut Shop,Creperie,Dance Studio,Deli / Bodega
3,Barangka Ilaya,14.572535,121.049159,0,Coffee Shop,Filipino Restaurant,Sandwich Place,Hotel,Multiplex,Dance Studio,Massage Studio,Chinese Restaurant,Café,Market
4,Barangka Itaas,14.570969,121.041918,0,Fast Food Restaurant,Tea Room,Filipino Restaurant,Bar,Diner,Burger Joint,Bookstore,Bed & Breakfast,Convenience Store,Tailor Shop
5,Buayang Bato,14.572752,121.053411,0,Coffee Shop,Sandwich Place,Filipino Restaurant,Japanese Restaurant,Yoga Studio,Bakery,Café,Chinese Restaurant,Dance Studio,Dessert Shop
6,Daang Bakal,14.593543,121.028065,0,Fast Food Restaurant,Convenience Store,Filipino Restaurant,Chinese Restaurant,Salon / Barbershop,Snack Place,Pharmacy,Bakery,Pizza Place,Supermarket
7,Hagdan Bato Itaas,14.588103,121.032641,0,Convenience Store,Fast Food Restaurant,Food Court,Coffee Shop,Diner,Shopping Mall,Burger Joint,Spa,Korean Restaurant,Café
8,Hagdan Bato Libis,14.588567,121.032967,0,Fast Food Restaurant,Convenience Store,Korean Restaurant,Café,Diner,Shopping Mall,Burger Joint,Coffee Shop,Spa,Breakfast Spot
9,Harapin Ang Bukas,14.590900,121.028056,0,Fast Food Restaurant,Chinese Restaurant,Café,Filipino Restaurant,Supermarket,Snack Place,Salon / Barbershop,Convenience Store,Donut Shop,Bookstore


### Plot the clusters of barangays/neighborhoods

In [119]:
# create map
Manda_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Manda_merged['Latitude'], Manda_merged['Longitude'], Manda_merged['Neighbourhood'], Manda_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Manda_map_clusters)
       
Manda_map_clusters

### Study the clusters

#### Cluster 1

In [116]:
Manda_merged.loc[Manda_merged['Cluster Labels'] == 0, Manda_merged.columns[[0] + list(range(4, Manda_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,Bar,Chinese Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Coffee Shop,Convenience Store,Yoga Studio,Movie Theater,BBQ Joint
1,Barangka Drive,Spa,BBQ Joint,Burger Joint,Filipino Restaurant,Massage Studio,Salon / Barbershop,Laundromat,Fast Food Restaurant,Convenience Store,Bar
2,Barangka Ibaba,Hotel,Bar,Taco Place,Basketball Court,Food Truck,Fast Food Restaurant,Donut Shop,Creperie,Dance Studio,Deli / Bodega
3,Barangka Ilaya,Coffee Shop,Filipino Restaurant,Sandwich Place,Hotel,Multiplex,Dance Studio,Massage Studio,Chinese Restaurant,Café,Market
4,Barangka Itaas,Fast Food Restaurant,Tea Room,Filipino Restaurant,Bar,Diner,Burger Joint,Bookstore,Bed & Breakfast,Convenience Store,Tailor Shop
5,Buayang Bato,Coffee Shop,Sandwich Place,Filipino Restaurant,Japanese Restaurant,Yoga Studio,Bakery,Café,Chinese Restaurant,Dance Studio,Dessert Shop
6,Daang Bakal,Fast Food Restaurant,Convenience Store,Filipino Restaurant,Chinese Restaurant,Salon / Barbershop,Snack Place,Pharmacy,Bakery,Pizza Place,Supermarket
7,Hagdan Bato Itaas,Convenience Store,Fast Food Restaurant,Food Court,Coffee Shop,Diner,Shopping Mall,Burger Joint,Spa,Korean Restaurant,Café
8,Hagdan Bato Libis,Fast Food Restaurant,Convenience Store,Korean Restaurant,Café,Diner,Shopping Mall,Burger Joint,Coffee Shop,Spa,Breakfast Spot
9,Harapin Ang Bukas,Fast Food Restaurant,Chinese Restaurant,Café,Filipino Restaurant,Supermarket,Snack Place,Salon / Barbershop,Convenience Store,Donut Shop,Bookstore


Cluster one consists mainly of fast food restuarants.

In [117]:
Manda_merged.loc[Manda_merged['Cluster Labels'] == 1, Manda_merged.columns[[0] + list(range(4, Manda_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Namayan,Trail,Outdoor Event Space,Basketball Court,Café,Yoga Studio,Donut Shop,Creperie,Dance Studio,Deli / Bodega,Department Store


Cluster 2 consitsts of mainly outdoor-related and sports venues.

In [118]:
Manda_merged.loc[Manda_merged['Cluster Labels'] == 2, Manda_merged.columns[[0] + list(range(4, Manda_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Mauway,Filipino Restaurant,Convenience Store,Burger Joint,Pet Store,Coffee Shop,Tennis Court,Food Court,Karaoke Bar,Yoga Studio,Creperie
14,New Zañiga,BBQ Joint,Convenience Store,Breakfast Spot,Supermarket,Filipino Restaurant,Food Court,Karaoke Bar,Beer Garden,Comfort Food Restaurant,Bar
15,Old Zañiga,BBQ Joint,Convenience Store,Breakfast Spot,Supermarket,Filipino Restaurant,Food Court,Karaoke Bar,Beer Garden,Comfort Food Restaurant,Bar
16,Wack-Wack Greenhills,Convenience Store,Hobby Shop,Auto Garage,Paper / Office Supplies Store,Chinese Restaurant,Diner,Creperie,Dance Studio,Deli / Bodega,Department Store


Cluster 3 consists of mainly convenience stores, BBQ joints, and supermarkets.

### Conclusion

Since my preference is catered by cluster 3. I will have now the following barangays as my options where to relocate.